In [1]:
import sys
import os
sys.path.append('..')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import glob

import utilities.functions as functions
from utilities.functions import (
    segmentacao_3_otimizada,
    retidos11,
    calcula_viabilidade_wide,
)

In [2]:
!pwd

/Users/maceli/ifood_case/code/notebook


In [3]:
!ls -la dados/gold/df_decil.parquet

ls: dados/gold/df_decil.parquet: No such file or directory


In [4]:
df = pd.read_parquet("dados/gold/df_decil.parquet")
df.head(10)


FileNotFoundError: [Errno 2] No such file or directory: 'dados/gold/df_decil.parquet'

In [ ]:
df['decil_1_num'] = df['decil_mes_2_1'].str.extract(r'(\d+)').astype(int)
df['decil_12_num'] = df['decil_mes_2_12'].str.extract(r'(\d+)').astype(int)
df['delta_decil'] = df['decil_1_num']-df['decil_12_num'] 

In [ ]:
df.columns

In [ ]:
# Clusters baseados em padrão de consumo
conditions_comportamento = [
    (df['total_amount_mes_1'] > df['total_amount_mes_12']),  # Aumentou gasto
    (df['total_amount_mes_1'] < df['total_amount_mes_12']),  # Reduziu gasto  
    (df['total_amount_mes_1'] == df['total_amount_mes_12'])  # Manteve
]
choices_comportamento = ["Crescimento", "Retração", "Estável"]
df['segmento_gasto'] = np.select(conditions_comportamento, choices_comportamento)

In [ ]:
import numpy as np

df['crescimento_amount'] = np.where(
    df['total_amount_mes_12'] == 0,
    0,
    (df['total_amount_mes_1'] - df['total_amount_mes_12']) / df['total_amount_mes_12']
)

df['crescimento_pedidos'] = (df['num_pedidos_mes_1'] - df['num_pedidos_mes_12']) / df['num_pedidos_mes_12']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, y='crescimento_amount', x='crescimento_pedidos', 
                hue='is_target', alpha=0.6, s=60)
plt.title('Crescimento: valor vs número de pedidos')
plt.xlabel('crescimento n° pedidos')

plt.show()

In [ ]:
p99 = df['crescimento_amount'].quantile(0.99)

acima_p99 = df[
    df['crescimento_amount'] > p99
]
acima_p99['customer_id'].nunique()

In [ ]:
acima_p99['crescimento_pedidos'].describe()
acima_p99['crescimento_amount'].mean()

In [ ]:
df['crescimento_pedidos'].mean()
df['crescimento_amount'].mean()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=acima_p99, y='crescimento_amount', x='crescimento_pedidos', 
                hue='is_target', alpha=0.6, s=60)
plt.title('Crescimento: valor vs número de pedidos P99')
plt.xlabel('crescimento n° pedidos')

plt.show()

In [ ]:
# Estatísticas básicas
amount_stats = df['crescimento_amount'].describe().to_dict()
print("Distribuição do crescimento:")
print(amount_stats)

# Estatísticas básicas
pedidos_stats = df['crescimento_pedidos'].describe().to_dict()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='num_pedidos_mes_12', y='total_amount_mes_12', 
                hue='is_target', alpha=0.6, s=60)

plt.title('Crescimento: Valor vs Número de Pedidos')
plt.xlabel('Crescimento N° Pedidos')
plt.ylabel('Crescimento Valor')
plt.show()

In [ ]:

crescimento_max = df['crescimento_amount'].quantile(0.99) 
df['crescimento_max'] = df['crescimento_amount'].clip(upper=crescimento_max)


features = ['total_amount_mes_1', 'num_pedidos_mes_1']
X = df[features]

from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X)

df['cluster'] = KMeans(n_clusters=3, random_state=42, n_init=10).fit_predict(X_scaled)
df.shape

In [ ]:
resumo = df.groupby('cluster').agg({
    'total_amount_mes_1': 'mean',
    'num_pedidos_mes_1': 'mean',
    'crescimento_max': 'mean',
    'customer_id': 'count'
}).round(2)

print("Perfil dos clusters:")
print(resumo)

In [ ]:
df00=df[(df['num_pedidos_mes_1']==1)& (df['num_pedidos_mes_12']==1)]
df11=df[df['num_pedidos_mes_1']<=10]
df12=df[df['num_pedidos_mes_1']>10]

In [ ]:
df00['total_amount_mes_1'].describe()

In [ ]:

df_1,clientes_retidos_1=retidos11(df11, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)


In [ ]:
resultados, agg = calcula_viabilidade_wide(df_1,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

In [ ]:
resultados, agg = calcula_viabilidade_wide(df_2,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

In [ ]:
resultados, agg = calcula_viabilidade_wide(df_3,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)